# Missing Boxes Loader 

In [ ]:
import os, math, overpass, json
import pandas as pd
import numpy as np
import urllib.request

from copy import deepcopy
from tqdm import tqdm_notebook, tqdm
from urllib.request import urlopen, HTTPError
from gsvloader import polygon_to_points, loader
from shapely.geometry import Polygon, Point, LineString, MultiLineString

keys = pd.read_csv('api-keys.csv', header=None)

In [ ]:
def linestring_to_coords(roads):
    point_entry = {
        "type": "FeatureCollection",
        "features": []
    }
    for road in tqdm(roads['features'],'converting linestrings to coordinates'):
        if (road['geometry']['type'] != 'LineString'):
            continue

        line = deepcopy(road['geometry']['coordinates'])
        road['geometry']['coordinates'] = []

        for i in range(len(line)-1):
            x1,y1 = line[i]
            x2,y2 = line[i+1]
            x1 += 0.0000000001

            degree = math.degrees(math.atan(abs((y1-y2)/(x1-x2))))

            if(degree < 45):
                if(x1 == min(x1,x2)): start_x,start_y, end_x,end_y = x1,y1, x2,y2
                else: start_x,start_y, end_x,end_y = x2,y2, x1,y1

                m = (start_y-end_y)/(start_x-end_x)
                FROM, TO = start_x, end_x
                cur_x, cur_y = start_x, start_y
                while(FROM < TO):
                    new_x, new_y = linearEquation_x(start_x, start_y, m, FROM)
                    dist = math.hypot(new_x-cur_x, new_y-cur_y)
                    meters = 111111*dist
                    if(meters > 50):
                        cur_x, cur_y = new_x, new_y
                        road['geometry']['coordinates'].append([round(cur_x, 8), round(cur_y, 8)])
                    FROM += 0.000000123
            else:
                if(y1 == min(y1,y2)): start_x,start_y, end_x,end_y = x1,y1, x2,y2
                else: start_x,start_y, end_x,end_y = x2,y2, x1,y1

                m = (start_y-end_y)/(start_x-end_x)
                FROM, TO = start_y, end_y
                cur_x, cur_y = start_x, start_y
                while(FROM < TO):
                    new_x, new_y = linearEquation_y(start_x, start_y, m, FROM)
                    dist = math.hypot(new_x-cur_x, new_y-cur_y)
                    meters = 111111*dist
                    if(meters > 50):
                        cur_x, cur_y = new_x, new_y
                        road['geometry']['coordinates'].append([round(cur_x, 8), round(cur_y, 8)])
                    FROM += 0.000000123
        point_entry['features'].append(road)
    return point_entry

def linearEquation_x(x1, y1, m, x):
    y = m*(x-x1)+y1
    return x,y

def linearEquation_y(x1, y1, m, y):
    x = (y - y1 + (m*x1)) / m
    return x,y

In [ ]:
with open(os.path.join('data','shapefiles','Nakhon-Si-Thammarat-en.geojson')) as f:
    data = json.load(f)

In [ ]:
num = 1
for subdistrict_ind, feature in enumerate(data['features']):
    print(subdistrict_ind)
    
    poly = Polygon(data['features'][subdistrict_ind]['geometry']['coordinates'][0])

    # radius of the Earth in km
    r = 6371000.0 

    Distance = 0.0009

    centerpoint = []

    geojson = {
        "features": [],
        "type": "FeatureCollection"
    }

    numblock = 2
    threshold = numblock * 100

    coordinate = []
    coordinate = np.array(feature['geometry']['coordinates'])
    block = []
    max_lat = np.amax(coordinate[:,:,1])
    min_lat = np.amin(coordinate[:,:,1])
    max_lng = np.amax(coordinate[:,:,0])
    min_lng = np.amin(coordinate[:,:,0])

    num_row_lat = int(np.ceil((max_lat-min_lat)/Distance))
    num_row_lng = int(np.ceil((max_lng-min_lng)/Distance))

    block = []
    for lng in range (num_row_lng):
        for lat in range (num_row_lat):
            block.append([
                [min_lng+(lng*Distance), min_lat+(lat*Distance)],
                [min_lng+(lng*Distance), min_lat+((lat+1)*Distance)],
                [min_lng+((lng+1)*Distance), min_lat+((lat+1)*Distance)],
                [min_lng+((lng+1)*Distance), min_lat+(lat*Distance)], 0
            ])

    for z in range(len(block)-1,0,-1):
        if(poly.contains(Point((block[z][0][0]+Distance/2),(block[z][0][1]+Distance/2))) == False):
            del block[z]

    totalBlock = len(block)

    prop = feature['properties']

    geojson_prop = {
        'PV_TN': prop['PV_TN'],
        'AP_TN': prop['AP_TN'],
        'TB_TN': prop['TB_TN']
    }

    entry = {
        'GEOJSON_PATH': os.path.join('data','shapefiles','Nakhon-Si-Thammarat-en.geojson'),
        'keys': np.squeeze(keys.values)
    }

    polygon = polygon_to_points.get_polygon(entry, geojson_prop)

    roads = polygon_to_points.generate_overpass_script(polygon)

    point_geojson = linestring_to_coords(roads)

    no_missing = []
    missing_area = deepcopy(point_geojson)
    for i, feature in enumerate(tqdm(point_geojson['features'], 'loading metadata')):

        missing_area['features'][i]['geometry']['coordinates'] = []

        for lng, lat in feature['geometry']['coordinates']:
            requestMeta = urlopen(
                "https://maps.googleapis.com/maps/api/streetview/metadata?"+\
                "location="+str(lat)+','+str(lng)+"&key="+entry['keys'][0]
            )

            metaJson = json.loads(requestMeta.read().decode('utf8'))
            if metaJson["status"] == 'OK':

                for k in range(totalBlock):
                    
                    minLngInBlock = block[k][0][0]
                    maxLngInBlock = block[k][2][0]
                    minLatInBlock = block[k][0][1]
                    maxLatInBlock = block[k][2][1]
                    
                    if((lng >= (minLngInBlock)) and lng <= (maxLngInBlock)):
                        if((lat >= (minLatInBlock)) and lat <= (maxLatInBlock)):
                            block[k][4] += 1


    for j in range(len(block)):
        if(block[j][4] == 0):
            centerpoint.append([block[j][0][0] + (Distance/2),block[j][0][1] + (Distance/2)])


    for l in range(len(block)):
        if(block[l][4] != 0):
            geojson['features'].append({
                "type": "Feature",
                "geometry": {
                    "coordinates": [block[l][:-1]],
                    "type": "Polygon"
                },
                "properties": {"fill": "#00aa22"}
            })

    for m in range((len(centerpoint)-1),0,-1):
        phi1 = radians(centerpoint[m][1])
        lamdha1 = radians(centerpoint[m][0])
        outOfArea = True

        for n in range (len(block)):
            if(block[n][4] != 0):
                coordinate = block[n][:-1]
                a = 3
                while a >= 0:
                    phi2 = radians(float(coordinate[a][1]))
                    lamdha2 = radians(float(coordinate[a][0]))
                    a = a-1

                    diff_phi = phi2 - phi1
                    diff_lamdha = lamdha2 - lamdha1

                    distance = 2 * r * asin(sqrt(pow(sin((diff_phi) / 2), 2) + cos(phi1) * cos(phi2) * pow(sin((diff_lamdha) / 2), 2)))

                    if(distance <= threshold):
                        outOfArea = False

        if(outOfArea == True or poly.contains(Point(centerpoint[m])) == False):
            del centerpoint[m]


    missingArea = []
    for p in range(len(centerpoint)):
        missingArea.append([[(centerpoint[p][0])-Distance/2,(centerpoint[p][1])-Distance/2],[(centerpoint[p][0])-Distance/2,(centerpoint[p][1])+Distance/2],[(centerpoint[p][0])+Distance/2,(centerpoint[p][1])+Distance/2],[(centerpoint[p][0])+Distance/2,(centerpoint[p][1])-Distance/2]])

    for q in range(len(missingArea)):
        geojson['features'].append({
            "type": "Feature",
                "geometry": {
                    "coordinates": [missingArea[q][:]],
                    "type": "Polygon"
                },
                "properties": {}
            })

    with open(os.path.join('data','missing-boxes',str(subdistrict_ind)+'.geojson'), 'w') as FILE:
        json.dump(geojson, FILE)
        
    print("===================================================")